<a href="https://colab.research.google.com/github/alirezakavianifar/gitTutorial/blob/developer/RLProject3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.logger import configure
import numpy as np
import os

class HealthcareNetworkEnv(gym.Env):
    def __init__(self, H, P, R, T, LeadTime, transport_costs, transshipment_costs, inventory_costs, ordering_costs):
        super(HealthcareNetworkEnv, self).__init__()

        # Define constants
        self.H = H  # Number of hospitals
        self.P = P  # Number of products
        self.R = R  # Number of suppliers
        self.T = T  # Number of periods
        self.LeadTime = LeadTime  # Lead time for orders

        # Costs
        self.transport_costs = transport_costs  # Cost of transporting products from suppliers to hospitals
        self.transshipment_costs = transshipment_costs  # Cost of transshipment between hospitals
        self.inventory_costs = inventory_costs  # Holding costs for inventory
        self.ordering_costs = ordering_costs  # Ordering costs

        # Define state space
        self.observation_space = spaces.Dict({
            'inventory': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32),
            'demand': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32),
            'supply_capacity': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32),
            'lead_time': spaces.Box(low=0, high=np.inf, shape=(R,), dtype=np.float32)
        })

        # Define action space as MultiDiscrete
        self.action_space = spaces.MultiDiscrete([10] * (H * P * R + H * H * P))

        # Initialize state
        self.state = self.reset()

    def reset(self, seed=None):
        self.state = {
            'inventory': np.zeros((self.H, self.P), dtype=np.float32),
            'demand': np.random.randint(0, 10, size=(self.H, self.P)).astype(np.float32),
            'supply_capacity': np.ones((self.H, self.P), dtype=np.float32),
            'lead_time': np.full((self.R,), self.LeadTime, dtype=np.float32)
        }
        return self.state, {}

    def step(self, action):
        # Extract order and transship actions from the MultiDiscrete action
        order_action_size = self.H * self.P * self.R
        order = np.array(action[:order_action_size]).reshape((self.H, self.P, self.R))
        transship = np.array(action[order_action_size:]).reshape((self.H, self.H, self.P))

        # Update inventory levels based on orders and transshipments
        self._update_inventory(order, transship)

        # Calculate rewards based on demand satisfaction and costs
        reward, demand_loss, costs = self._calculate_reward(order, transship)

        # Create a new state based on the updated inventory and new demand
        self._update_demand()
        next_state = self.state

        # Check if the episode is done (end of time period)
        done = self._check_done()

        return next_state, reward, False, done, {}

    def _update_inventory(self, order, transship):
        for h in range(self.H):
            for p in range(self.P):
                for r in range(self.R):
                    # Constraint 3: Capacity Constraint in Hospital h
                    # Ensure supply received by hospital h does not exceed its maximum capacity
                    supply_received = order[h, p, r]
                    capacity_available = self.state['supply_capacity'][h, p]
                    supply_received = min(supply_received, capacity_available)
                    self.state['inventory'][h, p] += supply_received

        # Process transshipments
        for h1 in range(self.H):
            for h2 in range(self.H):
                if h1 != h2:
                    for p in range(self.P):
                        # Constraint 8: Coverage Distance
                        # Ensure coverage distance is not exceeded
                        if transship[h1, h2, p] > 0:
                            coverage_distance = 1  # Assuming coverage distance is 1 for simplicity
                            if coverage_distance <= self.state['supply_capacity'][h1, p]:
                                self.state['inventory'][h1, p] -= transship[h1, h2, p]
                                self.state['inventory'][h2, p] += transship[h1, h2, p]

    def _calculate_reward(self, order, transship):
        reward = 0
        demand_loss = 0
        total_costs = 0

        for h in range(self.H):
            for p in range(self.P):
                for r in range(self.R):
                    # Constraints 4 and 5: Supplier Balance
                    # Ensure balance between ordered and received quantities
                    ordered_quantity = order[h, p, r]
                    received_quantity = self.state['inventory'][h, p]
                    if received_quantity < ordered_quantity:
                        demand_loss += (ordered_quantity - received_quantity)
                    else:
                        reward += received_quantity

        for h in range(self.H):
            for p in range(self.P):
                # Constraint 6: Supplier Balance
                # Ensure ordered quantities do not exceed supplier capacity
                for r in range(self.R):
                    ordered_quantity = order[h, p, r]
                    supplier_capacity = self.state['supply_capacity'][h, p]
                    if ordered_quantity > supplier_capacity:
                        ordered_quantity = supplier_capacity
                        order[h, p, r] = ordered_quantity

                # Constraints 7 and 8: Hospital Inventory Balance and Coverage Distance
                # Update hospital inventory and enforce coverage distance
                inventory_previous_period = self.state['inventory'][h, p]
                shortage = max(self.state['demand'][h, p] - inventory_previous_period, 0)
                transshipment_in = sum(transship[:, h, p])
                transshipment_out = sum(transship[h, :, p])
                inventory_next_period = inventory_previous_period + transshipment_in - transshipment_out - shortage
                if inventory_next_period < 0:
                    inventory_next_period = 0
                elif inventory_next_period > self.state['supply_capacity'][h, p]:
                    inventory_next_period = self.state['supply_capacity'][h, p]
                self.state['inventory'][h, p] = inventory_next_period

                # Constraint 8: Coverage Distance
                # Ensure coverage distance is not exceeded
                coverage_distance = 1  # Assuming coverage distance is 1 for simplicity
                if transshipment_out > 0 and coverage_distance * transshipment_out > self.state['supply_capacity'][h, p]:
                    transshipment_out = self.state['supply_capacity'][h, p] // coverage_distance
                transship[h, :,p] = transshipment_out

        # Calculate total costs and adjust reward
        # Add appropriate cost calculations

        return reward, demand_loss, total_costs

    def _update_demand(self):
        # Update demand based on the prediction horizon
        # This could be random or based on some defined patterns
        self.state['demand'] = np.random.randint(0, 10, size=(self.H, self.P)).astype(np.float32)

    def _check_done(self):
        # Define a condition to end the episode, e.g., end of a time period
        return False

# Define parameters
H = 5  # Number of hospitals
P = 1  # Number of products
R = 1  # Number of suppliers
T = 10  # Number of periods
LeadTime = 1  # Lead time for orders

# Define costs (for example purposes, using random values)
transport_costs = np.random.rand(R, H, P)
transshipment_costs = np.random.rand(H, H, P)
inventory_costs = np.random.rand(H, P)
ordering_costs = np.random.rand(P, H)

# Create the environment
env = HealthcareNetworkEnv(H, P, R, T, LeadTime, transport_costs, transshipment_costs, inventory_costs, ordering_costs)

# Check the environment
check_env(env)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:263: UserWarning: Your observation demand has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:263: UserWarning: Your observation inventory has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:263: UserWarning: Your observation supply_capacity has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [ ]:
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.logger import configure
import numpy as np
import os

class HealthcareNetworkEnv(gym.Env):
    def __init__(self, H, P, R, T, LeadTime, transport_costs, transshipment_costs, inventory_costs, ordering_costs):
        super(HealthcareNetworkEnv, self).__init__()

        # Define constants
        self.H = H  # Number of hospitals
        self.P = P  # Number of products
        self.R = R  # Number of suppliers
        self.T = T  # Number of periods
        self.LeadTime = LeadTime  # Lead time for orders, now a 2D array (H, P)

        # Costs
        self.transport_costs = transport_costs  # Cost of transporting products from suppliers to hospitals
        self.transshipment_costs = transshipment_costs  # Cost of transshipment between hospitals
        self.inventory_costs = inventory_costs  # Holding costs for inventory
        self.ordering_costs = ordering_costs  # Ordering costs

        # Define state space
        self.observation_space = spaces.Dict({
            'inventory': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32),
            'demand': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32),
            'supply_capacity': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32),
            'lead_time': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32)
        })

        # Define action space as MultiDiscrete
        self.action_space = spaces.MultiDiscrete([10] * (H * P * R + H * H * P))

        # Initialize state
        self.state = self.reset()

    def reset(self, seed=None):
        self.state = {
            'inventory': np.zeros((self.H, self.P), dtype=np.float32),
            'demand': np.random.randint(0, 10, size=(self.H, self.P)).astype(np.float32),
            'supply_capacity': np.ones((self.H, self.P), dtype=np.float32),
            'lead_time': self.LeadTime.astype(np.float32)
        }
        return self.state, {}

    def step(self, action):
        # Extract order and transship actions from the MultiDiscrete action
        order_action_size = self.H * self.P * self.R
        order = np.array(action[:order_action_size]).reshape((self.H, self.P, self.R))
        transship = np.array(action[order_action_size:]).reshape((self.H, self.H, self.P))

        # Update inventory levels based on orders and transshipments
        self._update_inventory(order, transship)

        # Calculate rewards based on demand satisfaction and costs
        reward, demand_loss, costs = self._calculate_reward(order, transship)

        # Create a new state based on the updated inventory and new demand
        self._update_demand()
        next_state = self.state

        # Check if the episode is done (end of time period)
        done = self._check_done()

        return next_state, reward, False, done, {}

    def _update_inventory(self, order, transship):
        for h in range(self.H):
            for p in range(self.P):
                for r in range(self.R):
                    # Constraint 3: Capacity Constraint in Hospital h
                    # Ensure supply received by hospital h does not exceed its maximum capacity
                    supply_received = order[h, p, r]
                    capacity_available = self.state['supply_capacity'][h, p]
                    supply_received = min(supply_received, capacity_available)
                    self.state['inventory'][h, p] += supply_received

        # Process transshipments
        for h1 in range(self.H):
            for h2 in range(self.H):
                if h1 != h2:
                    for p in range(self.P):
                        # Constraint 8: Coverage Distance
                        # Ensure coverage distance is not exceeded
                        if transship[h1, h2, p] > 0:
                            coverage_distance = 1  # Assuming coverage distance is 1 for simplicity
                            if coverage_distance <= self.state['supply_capacity'][h1, p]:
                                self.state['inventory'][h1, p] -= transship[h1, h2, p]
                                self.state['inventory'][h2, p] += transship[h1, h2, p]

    def _calculate_reward(self, order, transship):
        reward = 0
        demand_loss = 0
        total_costs = 0

        for h in range(self.H):
            for p in range(self.P):
                for r in range(self.R):
                    # Constraints 4 and 5: Supplier Balance
                    # Ensure balance between ordered and received quantities
                    ordered_quantity = order[h, p, r]
                    received_quantity = self.state['inventory'][h, p]
                    if received_quantity < ordered_quantity:
                        demand_loss += (ordered_quantity - received_quantity)
                    else:
                        reward += received_quantity

        for h in range(self.H):
            for p in range(self.P):
                # Constraint 6: Supplier Balance
                # Ensure ordered quantities do not exceed supplier capacity
                for r in range(self.R):
                    ordered_quantity = order[h, p, r]
                    supplier_capacity = self.state['supply_capacity'][h, p]
                    if ordered_quantity > supplier_capacity:
                        ordered_quantity = supplier_capacity
                        order[h, p, r] = ordered_quantity

                # Constraints 7 and 8: Hospital Inventory Balance and Coverage Distance
                # Update hospital inventory and enforce coverage distance
                inventory_previous_period = self.state['inventory'][h, p]
                shortage = max(self.state['demand'][h, p] - inventory_previous_period, 0)
                transshipment_in = sum(transship[:, h, p])
                transshipment_out = sum(transship[h, :, p])
                inventory_next_period = inventory_previous_period + transshipment_in - transshipment_out - shortage
                if inventory_next_period < 0:
                    inventory_next_period = 0
                elif inventory_next_period > self.state['supply_capacity'][h, p]:
                    inventory_next_period = self.state['supply_capacity'][h, p]
                self.state['inventory'][h, p] = inventory_next_period

                # Constraint 8: Coverage Distance
                # Ensure coverage distance is not exceeded
                coverage_distance = 1  # Assuming coverage distance is 1 for simplicity
                if transshipment_out > 0 and coverage_distance * transshipment_out > self.state['supply_capacity'][h, p]:
                    transshipment_out = self.state['supply_capacity'][h, p] // coverage_distance
                transship[h, :, p] = transshipment_out

        # Calculate total costs and adjust reward
        # Add appropriate cost calculations

        return reward, demand_loss, total_costs

    def _update_demand(self):
        # Update demand based on the prediction horizon
        # This could be random or based on some defined patterns
        self.state['demand'] = np.random.randint(0, 10, size=(self.H, self.P)).astype(np.float32)

    def _check_done(self):
        # Define a condition to end the episode, e.g., end of a time period
        return False

# Define parameters
H = 5  # Number of hospitals
P = 3  # Number of products
R = 1  # Number of suppliers
T = 10  # Number of periods

# Define lead times as a 2D array (H, P)
LeadTime = np.random.randint(1, 5, size=(H, P))

# Define costs (for example purposes, using random values)
transport_costs = np.random.rand(R, H, P)
transshipment_costs = np.random.rand(H, H, P)
inventory_costs = np.random.rand(H, P)
ordering_costs = np.random.rand(P, H)

# Create the environment
env = HealthcareNetworkEnv(H, P, R, T, LeadTime, transport_costs, transshipment_costs, inventory_costs, ordering_costs)

# Check the environment
check_env(env)


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:263: UserWarning: Your observation lead_time has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


<div dir="rtl"><h2>

## توضیحات  فضای حالت

</h2></div>

<div dir="rtl"><h3>


در اینجا کد فضای حالت (observation space) نشان داده شده است. در کد موجود، فرمت داده‌های مشاهده به صورت زیر تعریف شده است:


</h3></div>

```python
self.observation_space = spaces.Dict({
    'inventory': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32),
    'demand': spaces.Box(low=0, high=np.inf, shape=(H, P), dtype=np.float32),
    'supply_capacity': spaces.Box(low=0, high=1, shape=(H, P), dtype=np.float32),
    'lead_time': spaces.Box(low=0, high=np.inf, shape=(R,), dtype=np.float32)
})
```

<div dir="rtl"><h3>

- `inventory` (موجودی): این فضای مشاهده یک دیکشنری است که موجودی محصولات در هر بیمارستان را نشان می‌دهد. این فضا با `spaces.Box` تعریف شده است که به معنی یک جعبه (محدوده اعداد) است، که اعداد در آن باید بین ۰ و بی‌نهایت باشند (زیرا ممکن است موجودی همیشه برای یک محصول از همان مقدار تا بی‌نهایت تغییر کند). همچنین شکل آن `(H, P)` است که نشان‌دهنده تعداد بیمارستان‌ها و تعداد محصولات موجود در هر بیمارستان است.

- `demand` (تقاضا): این بخش نیز یک فضای مشاهده از نوع `spaces.Box` است که نشان‌دهنده تقاضا برای هر محصول در هر بیمارستان است. محدوده اعداد آن نیز بین ۰ و بی‌نهایت است و شکل آن نیز `(H, P)` است.

- `supply_capacity` (ظرفیت تأمین): این فضای مشاهده نیز یک `spaces.Box` است که نشان‌دهنده ظرفیت تأمین محصولات برای هر بیمارستان است. این بار محدوده اعداد بین ۰ و ۱ است، زیرا معمولاً ظرفیت تأمین به صورت نسبی نسبت به ظرفیت کل بیمارستان محاسبه می‌شود. شکل این فضا همچنین `(H, P)` است.

- `lead_time` (زمان سرباره): این بخش نیز یک `spaces.Box` است که نشان‌دهنده زمان سرباره برای هر تامین‌کننده است. این بخش فقط یک بعد دارد (با توجه به تنها بودن تأمین‌کننده در اینجا) و محدوده اعداد آن بین ۰ و بی‌نهایت است، زیرا زمان سرباره می‌تواند هر مقداری بین ۰ تا بی‌نهایت باشد.

این فضاهای مشاهده به صورت یک دیکشنری تعریف شده‌اند که هرکدام از کلیدهای آن (مانند `'inventory'` یا `'demand'`) یک نوع خاص از فضای مشاهده را نمایندگی می‌کنند. حالا با استفاده از یک مثال ساده، این مشاهدات را بررسی کنیم:

</h3></div>

```python
observation_example = {
    'inventory': np.array([[10.0], [5.0], [8.0], [3.0], [15.0]]),  # موجودی محصول در هر بیمارستان
    'demand': np.array([[8.0], [6.0], [9.0], [4.0], [7.0]]),  # تقاضا برای هر محصول در هر بیمارستان
    'supply_capacity': np.array([[0.8], [0.9], [0.7], [0.6], [0.5]]),  # ظرفیت تأمین برای هر محصول در هر بیمارستان
    'lead_time': np.array([1.0])  # زمان سرباره تامین‌کننده
}
```
<div dir="rtl"><h3>
در این مثال، هر بیمارستان یک محصول دارد و ابعاد آرایه‌ها را می‌توان با تعداد بیمارستان (H) و تعداد محصولات (P) توجیه کرد. همچنین تأمین‌کننده واحد (R=1) است. به علاوه، مقدار زمان لاگ برای تأمین‌کننده فقط یک مقدار است.

</h3></div>

<div dir="rtl"><h2>

## توضیحات  فضای اقدام

</h2></div>

<div dir="rtl"><h4>
فضای اقدام در کلاس `HealthcareNetworkEnv` به گونه‌ای طراحی شده است که پیچیدگی‌های سفارش محصولات از تأمین‌کنندگان و انتقال محصولات بین بیمارستان‌ها را مدیریت کند. در اینجا به تفصیل به منطق پشت تعریف و استفاده از فضای اقدام پرداخته می‌شود.
</h4></div>

<div dir="rtl"><h4>

### تعریف فضای اقدام:

</h4></div>

```python
self.action_space = spaces.MultiDiscrete([10] * (H * P * R + H * H * P))
```
<div dir="rtl"><h4>

این خط، فضای اقدام را به عنوان یک فضای `MultiDiscrete` تعریف می‌کند که هر عنصر در فضا می‌تواند یک مقدار گسسته بین 0 تا 9 (شامل) داشته باشد. تعداد کل اقدامات گسسته توسط عبارت `(H * P * R + H * H * P)` تعیین می‌شود، که در آن:

- `H`: تعداد بیمارستان‌ها.
- `P`: تعداد محصولات.
- `R`: تعداد تأمین‌کنندگان.

</h4></div>

<div dir="rtl"><h4>

### تفکیک فضای اقدام:

فضای اقدام شامل دو بخش اصلی است:

1. **اقدامات سفارش**:
   - نشان‌دهنده مقدار هر محصولی است که از هر تأمین‌کننده برای هر بیمارستان سفارش داده می‌شود.
   - تعداد کل اقدامات سفارش برابر است با `H * P * R`.

2. **اقدامات انتقال**:
   - نشان‌دهنده مقدار هر محصولی است که بین بیمارستان‌ها منتقل می‌شود.
   - تعداد کل اقدامات انتقال برابر است با `H * H * P`.

### ساختار فضای اقدام:

- **اقدامات سفارش**:
  - برای هر بیمارستان `h` (از 0 تا H-1):
    - برای هر محصول `p` (از 0 تا P-1):
      - برای هر تأمین‌کننده `r` (از 0 تا R-1):
        - اقدام نشان‌دهنده مقدار محصول `p` است که از تأمین‌کننده `r` به بیمارستان `h` سفارش داده می‌شود.

- **اقدامات انتقال**:
  - برای هر بیمارستان `h1` (از 0 تا H-1):
    - برای هر بیمارستان `h2` (از 0 تا H-1)، به غیر از `h1`:
      - برای هر محصول `p` (از 0 تا P-1):
        - اقدام نشان‌دهنده مقدار محصول `p` است که از بیمارستان `h1` به بیمارستان `h2` منتقل می‌شود.

</h4></div>

<div dir="rtl"><h4>

### استفاده از اقدامات:

هنگامی که تابع `step` با یک اقدام فراخوانی می‌شود، آرایه اقدام به دو بخش سفارش و انتقال تقسیم و متناسب بازسازی می‌شود:

</h4></div>

```python
order_action_size = self.H * self.P * self.R
order = np.array(action[:order_action_size]).reshape((self.H, self.P, self.R))
transship = np.array(action[order_action_size:]).reshape((self.H, self.H, self.P))
```

<div dir="rtl"><h4>

### مثال:

برای یک مثال ساده با `H = 2` بیمارستان، `P = 1` محصول، و `R = 1` تأمین‌کننده:

- **ابعاد کل اقدام**: `(H * P * R + H * H * P) = (2 * 1 * 1 + 2 * 2 * 1) = 6`
- **اقدامات سفارش**: دو عنصر اول نمایانگر مقدار سفارش از تأمین‌کننده به بیمارستان‌ها هستند.
  - `order[0]` مربوط به مقدار سفارش از تأمین‌کننده به بیمارستان 0 است.
  - `order[1]` مربوط به مقدار سفارش از تأمین‌کننده به بیمارستان 1 است.
- **اقدامات انتقال**: چهار عنصر بعدی نمایانگر مقدار انتقال بین بیمارستان‌ها هستند.
  - `transship[0][1]` و `transship[1][0]` مربوط به مقدار انتقال بین بیمارستان 0 و بیمارستان 1 است.

</h4></div>

<div dir="rtl"><h4>

### پیاده‌سازی در تابع Step:

1. **استخراج اقدامات سفارش و انتقال**:
   - سفارش‌ها و انتقال‌ها استخراج و بازسازی می‌شوند.

</h4></div>

   ```python
   order = np.array(action[:order_action_size]).reshape((self.H, self.P, self.R))
   transship = np.array(action[order_action_size:]).reshape((self.H, self.H, self.P))
   ```
<div dir="rtl"><h4>

2. **به‌روزرسانی موجودی**:
   - متد `_update_inventory` این اقدامات را پردازش کرده و موجودی را به‌روزرسانی می‌کند.

</h4></div>


   ```python
   self._update_inventory(order, transship)
   ```
<div dir="rtl"><h4>

3. **محاسبه پاداش‌ها**:
   - پاداش‌ها بر اساس اینکه اقدامات چقدر تقاضا را برآورده می‌کنند و هزینه‌های متحمل شده، محاسبه می‌شوند.

</h4></div>

   ```python
   reward, demand_loss, costs = self._calculate_reward(order, transship)
   ```

<div dir="rtl"><h4>

4. **بازگشت حالت جدید و پاداش**:
   - حالت جدید تولید شده و تابع step حالت به‌روزرسانی شده، پاداش و سایر اطلاعات را بازمی‌گرداند.



### نتیجه‌گیری:

فضای اقدام در کلاس `HealthcareNetworkEnv` به گونه‌ای طراحی شده است که یک مجموعه جامع از اقدامات را مدیریت کند که هم شامل سفارش محصولات از تأمین‌کنندگان و هم شامل انتقال محصولات بین بیمارستان‌ها می‌شود. این ساختار امکان شبیه‌سازی استراتژی‌های پیچیده مدیریت موجودی در یک شبکه بهداشتی را فراهم می‌کند.

</h4></div>

</h3></div>

<div dir="rtl"><h2>

## توضیحات تابع پاداش

</h2></div>

<div dir="rtl"><h3>

حالت پاداش را محاسبه می‌کنیم. ابتدا متغیرهای پاداش، از دست داده‌های تقاضا، و هزینه‌های کلی را صفر می‌کنیم.

</h3></div>

```python
reward = 0
demand_loss = 0
total_costs = 0
```
<div dir="rtl"><h3>

سپس در یک حلقه تعداد بیمارستان‌ها را پیمایش می‌کنیم.

</h3></div>

```python
for h in range(self.H):
```
<div dir="rtl"><h3>

در داخل آن، حلقه دیگری برای پیمایش تعداد محصولات موجود در هر بیمارستان است.

</h3></div>

```python
    for p in range(self.P):
```
<div dir="rtl"><h3>

و درون آن، یک حلقه برای تامین‌کنندگان به منظور بررسی موجودی‌های سفارش داده شده در هر بیمارستان را شروع می‌کنیم.

</h3></div>

```python
        for r in range(self.R):
```

<div dir="rtl"><h3>

حالا ما در محدودیت‌های ۴ و ۵ هستیم که مربوط به تعادل تامین‌کننده است و مطمئن می‌شویم که مقدار سفارش داده شده توسط تامین‌کننده برابر با مقدار دریافتی است.

</h3></div>

```python
            ordered_quantity = order[h, p, r]
            received_quantity = self.state['inventory'][h, p]
            if received_quantity < ordered_quantity:
                demand_loss += (ordered_quantity - received_quantity)
            else:
                reward += received_quantity
```
<div dir="rtl"><h3>

در ادامه، محدودیت ۶ را داریم که مربوط به تعادل تامین‌کننده است و مطمئن می‌شویم که مقدار سفارش داده شده توسط تامین‌کننده، ظرفیت تامین‌کننده را تجاوز نمی‌کند.

</h3></div>

```python
        for r in range(self.R):
            ordered_quantity = order[h, p, r]
            supplier_capacity = self.state['supply_capacity'][h, p]
            if ordered_quantity > supplier_capacity:
                ordered_quantity = supplier_capacity
                order[h, p, r] = ordered_quantity
```

<div dir="rtl"><h3>

در بخش بعدی، ما با محدودیت‌های ۷ و ۸ روبرو هستیم که به تعادل موجودی بیمارستان و فاصله پوشش مربوط است. موجودی بیمارستان را به‌روزرسانی کرده و فاصله پوشش را اعمال می‌کنیم.

</h3></div>

```python
        inventory_previous_period = self.state['inventory'][h, p]
        shortage = max(self.state['demand'][h, p] - inventory_previous_period, 0)
        transshipment_in = sum(transship[:, h, p])
        transshipment_out = sum(transship[h, :, p])
        inventory_next_period = inventory_previous_period + transshipment_in - transshipment_out - shortage
        if inventory_next_period < 0:
            inventory_next_period = 0
        elif inventory_next_period > self.state['supply_capacity'][h, p]:
            inventory_next_period = self.state['supply_capacity'][h, p]
        self.state['inventory'][h, p] = inventory_next_period

        coverage_distance = 1
        if transshipment_out > 0 and coverage_distance * transshipment_out > self.state['supply_capacity'][h, p]:
            transshipment_out = self.state['supply_capacity'][h, p] // coverage_distance
        transship[h, :, p] = transshipment_out
```
<div dir="rtl"><h3>

در نهایت، ما هزینه‌های کلی را محاسبه کرده و پاداش را تنظیم می‌کنیم.

</h3></div>

```python
# Calculate total costs and adjust reward
# Add appropriate cost calculations

return reward, demand_loss, total_costs
```
</h3></div>

<div dir="rtl"><h2>

## توضیحات تابع بروزرسانی موجودی

</h2></div>


<div dir="rtl">
<h3>


بخش به بخش تابع `_update_inventory` را توضیح می‌دهم:

1. این بخش ابتدا از سه حلقه برای حلقه زنی برای هر یک از بیمارستان‌ها، محصولات و تامین‌کنندگان استفاده می‌کند:



</h3></div>   

```python
def _update_inventory(self, order, transship):
    for h in range(self.H):
        for p in range(self.P):
            for r in range(self.R):
```


<div dir="rtl"><h3>

2. سپس، سفارش‌هایی که از تامین‌کنندگان دریافت شده وارد سیستم می‌شوند و موجودی انبار مربوط به هر بیمارستان افزایش می‌یابد:



</h3></div>  

```python
# Constraint 3: Capacity Constraint in Hospital h
# Ensure supply received by hospital h does not exceed its maximum capacity
supply_received = order[h, p, r]
capacity_available = self.state['supply_capacity'][h, p]
supply_received = min(supply_received, capacity_available)
self.state['inventory'][h, p] += supply_received
```


<div dir="rtl"><h3>

3. در این قسمت، حلقه‌ها برای پردازش انتقالات انجام می‌شوند. اگر تراکنش مثبت بود (به معنای انتقال محصول از بیمارستان h1 به h2)، این بخش اطمینان حاصل می‌کند که انتقال به مقدار در دسترس است:


</h3></div>  

```python
# Process transshipments
for h1 in range(self.H):
    for h2 in range(self.H):
        if h1 != h2:
            for p in range(self.P):
                # Constraint 8: Coverage Distance
                # Ensure coverage distance is not exceeded
                if transship[h1, h2, p] > 0:
                    coverage_distance = 1  # Assuming coverage distance is 1 for simplicity
                    if coverage_distance <= self.state['supply_capacity'][h1, p]:
                        self.state['inventory'][h1, p] -= transship[h1, h2, p]
                        self.state['inventory'][h2, p] += transship[h1, h2, p]
```


<div dir="rtl"><h3>

این تابع در واقع موجودی انبار را بروزرسانی می‌کند تا مطمئن شود تامین محصولات مورد نیاز بیمارستان‌ها از تامین‌کنندگان صورت گرفته و انتقالات بین بیمارستان‌ها به درستی انجام شود.


</h3></div>